# Script to make cpm counts matrix for deconvolution


In [41]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from collections import defaultdict
import os
from matplotlib import rcParams
import re


In [42]:
meta1 = pd.read_excel("../Gates_metadata.xlsx", sheet_name=1)
meta1["sample"] = "SAMPLE_" + meta1["PID"].astype(str) + 14*["_1", "_2", "_3"]
meta1.set_index("sample", inplace=True)

In [43]:
meta1["other_name"] = meta1["PID"].astype(str) + "_" + meta1["visit code"].astype(str)
meta1["old_name"] = meta1.index
m = meta1.set_index("other_name")

In [44]:
# Load gene counts table
genes = pd.read_table("../gene_counts.txt", index_col=0).T

genes_duprm = genes.copy().loc[[s for s in genes.index if "duprm" in s]]
genes_duprm["sample"] = [s.split(".")[0] for s in genes_duprm.index]
genes_duprm = genes_duprm.groupby("sample").sum().T

genes = genes.copy().loc[[s for s in genes.index if "duprm" not in s]]
genes["sample"] = [s.split(".")[0] for s in genes.index]
genes = genes.groupby("sample").sum().T

In [45]:
just_genes = genes.iloc[:-5]
just_genes = genes.loc[[g for g in genes.index if "ENSG" in g]]
cpm = (1e6*just_genes.div(just_genes.sum()))
logCPM = np.log10(cpm+1)

X = logCPM.loc[:, meta1.index.difference(["SAMPLE_100200_1"])].T

In [46]:
df = X.T.copy()
df.head(4)

sample,SAMPLE_100008_1,SAMPLE_100008_2,SAMPLE_100008_3,SAMPLE_100137_1,SAMPLE_100137_2,SAMPLE_100137_3,SAMPLE_100177_1,SAMPLE_100177_2,SAMPLE_100177_3,SAMPLE_100200_2,...,SAMPLE_100261_3,SAMPLE_100267_1,SAMPLE_100267_2,SAMPLE_100267_3,SAMPLE_100306_1,SAMPLE_100306_2,SAMPLE_100306_3,SAMPLE_100337_1,SAMPLE_100337_2,SAMPLE_100337_3
gene,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,0.108051,0.134956,0.000000,0.000000,0.017846,0.034439,0.000000,0.059528,0.750694,0.000000,...,0.470290,0.000000,0.000000,0.070308,0.079665,0.252987,0.064509,0.000000,0.032014,0.000000
ENSG00000000005,0.000000,0.059101,0.000000,0.000000,0.000000,0.000000,0.000000,0.010505,0.000000,0.000000,...,0.192582,0.000000,0.000000,0.000000,0.021328,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000419,0.652799,0.636348,1.008460,0.363420,0.254551,0.880937,0.832677,0.691747,0.794950,0.576672,...,1.213423,0.481709,0.715704,0.772355,0.897398,0.605353,0.743287,0.575082,1.216245,0.058721
ENSG00000000457,0.649374,0.465296,0.980569,0.330649,0.217535,0.661805,0.971968,0.669566,1.499882,0.485037,...,1.123104,0.481709,1.078143,0.516469,0.681264,0.541761,1.244210,0.336793,0.676556,0.892349


In [47]:
# Load gene names and HIV associated genes
ensembl = pd.read_table("../ensembl.tsv", index_col=0)
ensembl_clean = ensembl[ensembl["Gene name"].apply(lambda s: len(s.split("-"))) == 1]

In [48]:
ensembl = ensembl_clean.reset_index().rename(columns={'Gene stable ID':'gene'}).copy()


In [49]:
df = df.merge(ensembl, on = 'gene').copy()

In [50]:
df.set_index('Gene name', inplace=True)


In [51]:
df = df.drop(['gene'],axis = 1).copy()

In [55]:
df.iloc[:,[0]].to_csv('gene_names_x_samples_100006_1.csv', index = 'True')

In [38]:
df.to_csv('gene_names_x_samples.csv', index = 'True')

In [39]:
thuyDat = pd.read_csv('gene_names_x_samples.csv', sep = ",", index_col = 0)

In [40]:
thuyDat

,SAMPLE_100008_1,SAMPLE_100008_2,SAMPLE_100008_3,SAMPLE_100137_1,SAMPLE_100137_2,SAMPLE_100137_3,SAMPLE_100177_1,SAMPLE_100177_2,SAMPLE_100177_3,SAMPLE_100200_2,...,SAMPLE_100261_3,SAMPLE_100267_1,SAMPLE_100267_2,SAMPLE_100267_3,SAMPLE_100306_1,SAMPLE_100306_2,SAMPLE_100306_3,SAMPLE_100337_1,SAMPLE_100337_2,SAMPLE_100337_3
Gene name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0.108051,0.134956,0.000000,0.000000,0.017846,0.034439,0.000000,0.059528,0.750694,0.000000,...,0.470290,0.000000,0.000000,0.070308,0.079665,0.252987,0.064509,0.000000,0.032014,0.000000
TNMD,0.000000,0.059101,0.000000,0.000000,0.000000,0.000000,0.000000,0.010505,0.000000,0.000000,...,0.192582,0.000000,0.000000,0.000000,0.021328,0.000000,0.000000,0.000000,0.000000,0.000000
DPM1,0.652799,0.636348,1.008460,0.363420,0.254551,0.880937,0.832677,0.691747,0.794950,0.576672,...,1.213423,0.481709,0.715704,0.772355,0.897398,0.605353,0.743287,0.575082,1.216245,0.058721
SCYL3,0.649374,0.465296,0.980569,0.330649,0.217535,0.661805,0.971968,0.669566,1.499882,0.485037,...,1.123104,0.481709,1.078143,0.516469,0.681264,0.541761,1.244210,0.336793,0.676556,0.892349
C1orf112,0.553597,0.468895,0.375521,0.459326,0.427797,0.449570,1.206147,0.801927,1.011351,0.516342,...,0.609518,0.631670,0.736333,0.480137,0.456727,0.447267,0.531737,0.561780,0.061829,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC008993.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AL512428.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AC008993.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
